In [1]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [2]:
import tensorflow as tf
from typing import Any
from pathlib import Path
import xml.etree.ElementTree as ET

2026-01-14 17:57:40.946420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-14 17:57:41.000699: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-14 17:57:41.012762: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-14 17:57:41.095861: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-14 17:57:42.992095: W tensorflow/compiler/tf2

In [3]:
from mobilenetv2ssd.core.config import load_config

In [4]:
main_cfg_path = "configs/train/default.yaml"
model_cfg_path = "configs/model/mobilenetv2_ssd_voc.yaml"
data_cfg_path = "configs/data/voc_224.yaml"
eval_cfg_path = "configs/eval/default.yaml"

In [5]:
config = load_config(main_cfg_path,model_cfg_path,data_cfg_path,eval_cfg_path)

In [6]:
from datasets.voc import build_voc_dataset
from datasets.base import BaseDetectionDataset
from datasets.transforms import build_train_transforms

In [7]:
compose = build_train_transforms(config)

In [8]:
data = build_voc_dataset(config, split = "train", transform = compose)

In [9]:
data[1]

I0000 00:00:1768431465.583104    7289 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1768431465.904502    7289 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1768431465.904583    7289 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1768431465.906456    7289 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1768431465.906554    7289 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

(<tf.Tensor: shape=(300, 300, 3), dtype=float32, numpy=
 array([[[0.48604575, 0.5605556 , 0.5853922 ],
         [0.50652945, 0.5771177 , 0.6241765 ],
         [0.52599347, 0.5965817 , 0.65148365],
         ...,
         [0.39251542, 0.38793448, 0.45823443],
         [0.4305098 , 0.40107843, 0.49433333],
         [0.445477  , 0.41233975, 0.516928  ]],
 
        [[0.5075948 , 0.5821046 , 0.6073726 ],
         [0.5202353 , 0.59082353, 0.6365883 ],
         [0.53658175, 0.60717   , 0.6607778 ],
         ...,
         [0.3982612 , 0.40587556, 0.46994063],
         [0.4092157 , 0.41637257, 0.4922157 ],
         [0.41096732, 0.41812417, 0.49724182]],
 
        [[0.5250327 , 0.599183  , 0.6277124 ],
         [0.5354902 , 0.60607845, 0.64529413],
         [0.54784316, 0.6184314 , 0.66333336],
         ...,
         [0.400033  , 0.41238564, 0.46918315],
         [0.38617647, 0.40754902, 0.46598038],
         [0.38356212, 0.40385625, 0.46552292]],
 
        ...,
 
        [[0.38843116, 0.45117626

In [10]:
def generator(dataset):
    for index in range(len(dataset)):
        image, target = dataset[index]
        yield {
            'image': image,
            'boxes': target['boxes'],
            'labels': target['labels'],
            'path': target['path'],
            'image_id': target['image_id'],
            'hash_signature': target['hash_signature'],
        }

In [11]:
output_signature = {
    "image": tf.TensorSpec(shape=(None, None, 3), dtype=tf.float32),
    "boxes": tf.TensorSpec(shape=(None, 4), dtype=tf.float32),
    "labels": tf.TensorSpec(shape=(None,), dtype=tf.int32),
    "path": tf.TensorSpec(shape=(None), dtype=tf.string),
    "image_id": tf.TensorSpec(shape=(None), dtype=tf.string),
    "hash_signature": tf.TensorSpec(shape=(None), dtype=tf.string),
}

In [12]:
dataset = tf.data.Dataset.from_generator(data._generator,output_signature = output_signature)

In [13]:
config['data']['train']

{'batch_size': 3,
 'padded_shapes': {'image': [300, 300, 3],
  'boxes': [None, 4],
  'labels': [None],
  'image_id': [],
  'hash_signature': [],
  'path': []},
 'padding_values': {'image': 0.0, 'boxes': -1.0, 'labels': 0},
 'shuffle': False,
 'prefetch': True,
 'max_boxes': 100,
 'repeat': True,
 'transform': {'enabled': True,
  'output_box_norm': False,
  'pipeline': ['photometric_distort',
   'random_expand',
   'random_iou_crop',
   'random_flip'],
  'params': {'random_flip': {'enabled': False,
    'prob': 0.5,
    'direction': 'horizontal'},
   'random_iou_crop': {'enabled': False,
    'prob': 1.0,
    'min_iou_choices': [0.1, 0.3, 0.5, 0.7, 0.9, None],
    'min_scale': 0.3,
    'max_scale': 1.0,
    'max_attempts': 50,
    'fallback': 'original'},
   'random_expand': {'enabled': False,
    'prob': 0.5,
    'max_ratio': 4.0,
    'fill': 'mean',
    'value': [0.485, 0.456, 0.406]},
   'photometric_distort': {'enabled': False,
    'prob': 0.5,
    'brightness': 0.125,
    'contrast':

## Builder Functions

In [14]:
def create_training_dataset_config(config: dict[str, Any]):
    training_dataset_opts = config['data'].get('train', {})

    dataset_opts = {
        'batch_size': training_dataset_opts.get('batch_size', 5),
        'padded_shapes': {
            'boxes': training_dataset_opts.get('padded_shapes', {}).get('boxes', [None, 4]),
            'image': training_dataset_opts.get('padded_shapes', {}).get('image', [224,224, 3]),
            'labels': training_dataset_opts.get('padded_shapes', {}).get('labels', [None]),
            'image_id': training_dataset_opts.get('padded_shapes', {}).get('image_id', []),
            'hash_signature': training_dataset_opts.get('padded_shapes', {}).get('hash_signature', []),
            'path': training_dataset_opts.get('padded_shapes', {}).get('path', []),
        },
        'padding_values': {
            'image': training_dataset_opts.get('padding_values', {}).get('image', 0.0),
            'boxes': training_dataset_opts.get('padding_values', {}).get('boxes', -1.0),
            'labels': training_dataset_opts.get('padding_values', {}).get('labels', -1),
        },
        'shuffle': training_dataset_opts.get('shuffle', False),
        'prefetch': training_dataset_opts.get('prefetch', False),
        'max_boxes': training_dataset_opts.get('max_boxes', 100),
        'repeat': training_dataset_opts.get('repeat', False),
    }

    return dataset_opts

In [15]:
def create_validation_dataset_config(config: dict[str, Any]):
    validation_dataset_opts = config['data'].get('val', {})

    dataset_opts = {
        'batch_size': validation_dataset_opts.get('batch_size', 5),
        'padded_shapes': {
            'boxes': validation_dataset_opts.get('padded_shapes', {}).get('boxes', [None, 4]),
            'image': validation_dataset_opts.get('padded_shapes', {}).get('image', [224,224, 3]),
            'labels': validation_dataset_opts.get('padded_shapes', {}).get('labels', [None]),
            'image_id': validation_dataset_opts.get('padded_shapes', {}).get('image_id', []),
            'hash_signature': validation_dataset_opts.get('padded_shapes', {}).get('hash_signature', []),
            'path': validation_dataset_opts.get('padded_shapes', {}).get('path', []),
        },
        'padding_values': {
            'image': validation_dataset_opts.get('padding_values', {}).get('image', 0.0),
            'boxes': validation_dataset_opts.get('padding_values', {}).get('boxes', -1.0),
            'labels': validation_dataset_opts.get('padding_values', {}).get('labels', -1),
        },
        'shuffle': validation_dataset_opts.get('shuffle', False),
        'prefetch': validation_dataset_opts.get('prefetch', False),
        'max_boxes': validation_dataset_opts.get('max_boxes', 100),
    }

    return dataset_opts

In [16]:
create_validation_dataset_config(config)

{'batch_size': 16,
 'padded_shapes': {'boxes': [None, 4],
  'image': [300, 300, 3],
  'labels': [None],
  'image_id': [],
  'hash_signature': [],
  'path': []},
 'padding_values': {'image': 0.0, 'boxes': -1.0, 'labels': 0},
 'shuffle': False,
 'prefetch': True,
 'max_boxes': 100}

In [17]:
create_training_dataset_config(config)

{'batch_size': 3,
 'padded_shapes': {'boxes': [None, 4],
  'image': [300, 300, 3],
  'labels': [None],
  'image_id': [],
  'hash_signature': [],
  'path': []},
 'padding_values': {'image': 0.0, 'boxes': -1.0, 'labels': 0},
 'shuffle': False,
 'prefetch': True,
 'max_boxes': 100,
 'repeat': True}

## Factory Dataset Creators

In [18]:
def _create_gt_mask(data):
    gt_mask = data['labels'] > 0
    data['valid_gt_mask'] = gt_mask
    return data

In [19]:
def create_validation_dataset(config: dict[str, Any], dataset: BaseDetectionDataset):
    dataset_opts = create_validation_dataset_config(config)

    tf_dataset = tf.data.Dataset.from_generator(dataset._generator,output_signature = output_signature)

    # Now checking for the options
    if dataset_opts['shuffle']:
        tf_dataset = tf_dataset.shuffle(buffer_size = 1000, reshuffle_each_iteration=True)

    # Padding the batch is crucial for the dataset
    tf_dataset = tf_dataset.padded_batch(batch_size = dataset_opts['batch_size'], padded_shapes = dataset_opts['padded_shapes'], padding_values = {
        'boxes' : tf.constant(dataset_opts['padding_values']['boxes'], tf.float32),
        'image' : tf.constant(dataset_opts['padding_values']['image'], tf.float32),
        'labels' : tf.constant(dataset_opts['padding_values']['labels'], tf.int32),
        'image_id' : tf.constant('', tf.string),
        'hash_signature' : tf.constant('', tf.string),
        'path' : tf.constant('', tf.string),
    })

    # Mapping a valid mask function
    tf_dataset = tf_dataset.map(_create_gt_mask, num_parallel_calls = tf.data.AUTOTUNE)

    # Adding prefetch
    if dataset_opts['prefetch']:
        tf_dataset = tf_dataset.prefetch(tf.data.AUTOTUNE)
    
    return tf_dataset

In [20]:
def create_training_dataset(config: dict[str, Any], dataset: BaseDetectionDataset):
    dataset_opts = create_training_dataset_config(config)

    tf_dataset = tf.data.Dataset.from_generator(dataset._generator,output_signature = output_signature)

    # Now checking for the options
    if dataset_opts['shuffle']:
        tf_dataset = tf_dataset.shuffle(buffer_size = 1000, reshuffle_each_iteration=True)

    if dataset_opts['repeat']:
        tf_dataset = tf_dataset.repeat() # Indefinite repeat so the dataset at the end is not hindered after one epoch

    # Padding the batch is crucial for the dataset
    tf_dataset = tf_dataset.padded_batch(batch_size = dataset_opts['batch_size'], padded_shapes = dataset_opts['padded_shapes'], padding_values = {
        'boxes' : tf.constant(dataset_opts['padding_values']['boxes'], tf.float32),
        'image' : tf.constant(dataset_opts['padding_values']['image'], tf.float32),
        'labels' : tf.constant(dataset_opts['padding_values']['labels'], tf.int32),
        'image_id' : tf.constant('', tf.string),
        'hash_signature' : tf.constant('', tf.string),
        'path' : tf.constant('', tf.string),
    })

    # Mapping a valid mask function
    tf_dataset = tf_dataset.map(_create_gt_mask, num_parallel_calls = tf.data.AUTOTUNE)

    # Adding prefetch
    if dataset_opts['prefetch']:
        tf_dataset = tf_dataset.prefetch(tf.data.AUTOTUNE)
    
    return tf_dataset

In [21]:
dataset = create_training_dataset(config, data)

In [22]:
val_dataset = create_validation_dataset(config, data)

In [23]:
for info in val_dataset.take(1):
    print(info)

{'image': <tf.Tensor: shape=(16, 300, 300, 3), dtype=float32, numpy=
array([[[[1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         ...,
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00]],

        [[1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         ...,
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00]],

        [[1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
         ...,


2026-01-14 17:57:59.191322: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [24]:
len(data)

11540